In [14]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import sys
from collections import Counter
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import multiprocessing
from multiprocessing import Pool
import itertools
from copy import deepcopy
import random

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, PReLU, GlobalMaxPooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical, Sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.resnet_v2 import ResNet152V2, preprocess_input
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121, DenseNet169
import tensorflow as tf
from skimage.transform import AffineTransform, warp

from datetime import datetime
import pickle
import scipy.stats as stats
import os
import sklearn.metrics
import tensorflow_hub as hub

from efficientnet.keras import EfficientNetB5
from keras_squeeze_excite_network.se_resnext import SEResNextImageNet as SEResNext

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
y = pd.read_csv("data/train.csv")
yEval = pd.read_csv("data/test.csv")
classMap = pd.read_csv("data/class_map.csv")

In [4]:
labels = ["grapheme_root","vowel_diacritic","consonant_diacritic"]

In [5]:
y = y.set_index("image_id")

In [6]:
tables = [pq.read_table('data/train_image_data_{0}.parquet'.format(i)) for i in range(4)]
tables = [table.to_pandas() for table in tables]
df = pd.concat(tables)
df = df.set_index("image_id")
del tables

In [7]:
def affine_image(img):
    """

    Args:
        img: (h, w) or (1, h, w)

    Returns:
        img: (h, w)
    """
    # ch, h, w = img.shape
    # img = img / 255.
    if img.ndim == 3:
        img = img[0]

    # --- scale ---
    min_scale = 0.8
    max_scale = 1.2
    sx = np.random.uniform(min_scale, max_scale)
    sy = np.random.uniform(min_scale, max_scale)

    # --- rotation ---
    max_rot_angle = 7
    rot_angle = np.random.uniform(-max_rot_angle, max_rot_angle) * np.pi / 180.

    # --- shear ---
    max_shear_angle = 10
    shear_angle = np.random.uniform(-max_shear_angle, max_shear_angle) * np.pi / 180.

    # --- translation ---
    max_translation = 4
    tx = np.random.randint(-max_translation, max_translation)
    ty = np.random.randint(-max_translation, max_translation)

    tform = AffineTransform(scale=(sx, sy), rotation=rot_angle, shear=shear_angle,
                            translation=(tx, ty))
    transformed_image = warp(img, tform)
    assert transformed_image.ndim == 2
    return (transformed_image*255).astype(np.uint8)

In [8]:
size=(120, 120)

def calcRotate(img):
    detector = cv2.ORB_create()
    keypoints=detector.detect(img)
    descriptors=detector.compute(img,keypoints)
    angles=[]
    weights=[]
    for i in descriptors[0]:
        if i.angle!=-1:
            angles.append(i.angle)
            weights.append(i.response)
    if len(angles)==0:
        return 0
    else:
        return np.average(angles,weights=weights)

def upper(img,mergin=20):
    up=0
    bottom=img.shape[0]
    while bottom-up>2:
        mid=(up+bottom)//2
        if np.sum(img[up:mid,:])==0:
            up=mid
        else:
            bottom=mid
    return max(up-mergin,0)

def lower(img,mergin=20):
    up=0
    bottom=img.shape[0]
    while bottom-up>2:
        mid=(up+bottom)//2
        if np.sum(img[mid:bottom,:])==0:
            bottom=mid
        else:
            up=mid
    return min(bottom+mergin,img.shape[0])

def lefter(img,mergin=20):
    left=0
    right=img.shape[1]
    while right-left>2:
        mid=(left+right)//2
        if np.sum(img[:,left:mid])==0:
            left=mid
        else:
            right=mid
    return max(left-mergin,0)

def righter(img,mergin=20):
    left=0
    right=img.shape[1]
    while right-left>2:
        mid=(left+right)//2
        if np.sum(img[:,mid:right])==0:
            right=mid
        else:
            left=mid
    return min(right+mergin,img.shape[1])

def transformImg(img,size=(255,255),training=True):
    ret2, img = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
    img = 255-img
    if training:
        img=affine_image(img)
    img = img[upper(img):lower(img),lefter(img):righter(img)]
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.resize(img, (size[0],size[1]))
    kernel = np.ones((3,3),np.float32)/9
    img = cv2.filter2D(img,-1,kernel)
    return img

def randomErase(img, prob=True):
    # random erasing
    # https://github.com/yu4u/cutout-random-erasing
    p = 0.5
    s_l = 0.02
    s_h = 0.4
    r_1 = 0.3
    r_2 = 1 / 0.3
    v_l = 0
    v_h = 255
    input_size=size[0]
    if prob==False or np.random.random()<p:

        while True:
            s = np.random.uniform(s_l, s_h) * input_size * input_size
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, input_size)
            top = np.random.randint(0, input_size)
            if left + w <= input_size and top + h <= input_size:
                break
        c = np.random.uniform(v_l, v_h, (h, w, 3))
        img[top : top + h, left : left + w, :] = c
    return img

In [9]:
class DataLoader(Sequence):
    def __init__(self,X,y,training,batch_size=64,size=(255,255),alpha=3):
        self.training = training
        self.batch_size=batch_size
        self.X=X
        self.y=y
        self.size=size
        self.alpha=alpha

    def __len__(self):
        return int(np.ceil(self.X.shape[0] / self.batch_size))


    def __getitem__(self, idx):
        
        _imgs=self.X[idx * self.batch_size:(idx + 1) * self.batch_size,:,:]
        
            
        
        imgs=[]
        for img in _imgs:
            imgs.append(transformImg(img,size=self.size,training=self.training))
         
        
        
        imgs=np.asarray(imgs)
        
        
        ret_y=[]
        for label in labels:
            ret_y.append(to_categorical(self.y[idx * self.batch_size:(idx + 1) * self.batch_size][label],num_classes=len(set(y[label]))))
    
    
        #mix up
        if self.training :
            r = np.random.permutation(imgs.shape[0])
            imgs2=deepcopy(imgs)[r]
            grapheme=ret_y[0]
            vowel=ret_y[1]
            consonant=ret_y[2]
            grapheme2=deepcopy(grapheme)[r]
            vowel2=deepcopy(vowel)[r]
            consonant2=deepcopy(consonant)[r]
            ratio=np.random.beta(self.alpha,self.alpha,imgs.shape[0])
            ratio[ratio>1]=1
            ratio[ratio<0]=0
            imgs=np.tile(ratio,(3,*size,1)).T*imgs+np.tile((1-ratio),(3,*size,1)).T*imgs2
            grapheme=np.tile(ratio,(168,1)).T*grapheme+np.tile((1-ratio),(168,1)).T*grapheme2
            vowel=np.tile(ratio,(11,1)).T*vowel+np.tile((1-ratio),(11,1)).T*vowel2
            consonant=np.tile(ratio,(7,1)).T*consonant+np.tile((1-ratio),(7,1)).T*consonant2
            grapheme=grapheme.astype(np.float32)
            vowel=vowel.astype(np.float32)
            consonant=consonant.astype(np.float32)
            ret_y=[grapheme,vowel,consonant]
   
        if self.training:
            #imgs = [randomErase(img) for img in imgs]
            pass
            
            
        imgs = np.asarray(imgs).astype(np.float32)/255.0
            

        return imgs, ret_y



In [10]:
def getMultiBased():
    model =  EfficientNetB5(weights="imagenet", include_top=False, input_shape=(*size, 3))
    x = model.output  
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    #x = Dense(1024, activation="relu")(x)
    grapheme = Dense(168, activation="softmax")(x)
    vowel = Dense(11,activation="softmax")(x)
    consonant = Dense(7, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=[grapheme,vowel,consonant])
    return model

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df.values.reshape(-1,137,236), y, train_size=0.9, random_state=8000)
del df

In [12]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=20)
checkpoint = ModelCheckpoint(filepath="tmp-eff5-epoch{epoch:04}.h5")

In [ ]:
batch_size=64
train_gen = DataLoader(X_train, y_train, training=True, batch_size=batch_size, size=size)
valid_gen = DataLoader(X_test, y_test, training=False, batch_size=batch_size, size=size)
#model = getMultiBased()
#model.summary()
#model.compile(optimizer=Adam(), metrics=["acc"], loss="categorical_crossentropy", loss_weights=[0.5,0.25,0.25])
print("compiled")
#model=load_model("tmp-eff5-epoch0045.h5")
model.fit_generator(train_gen, validation_data=valid_gen, epochs=80, callbacks=[checkpoint], workers=64, use_multiprocessing=True)
model.save("multiEfficientNetB5.h5")

compiled
Epoch 1/80
2825/2825 [==============================] - 1574s 557ms/step - loss: 0.7154 - dense_1_loss: 0.8978 - dense_2_loss: 0.6320 - dense_3_loss: 0.4340 - dense_1_acc: 0.7566 - dense_2_acc: 0.8679 - dense_3_acc: 0.9057 - val_loss: 0.0565 - val_dense_1_loss: 0.1570 - val_dense_2_loss: 0.0548 - val_dense_3_loss: 0.0470 - val_dense_1_acc: 0.9580 - val_dense_2_acc: 0.9865 - val_dense_3_acc: 0.9881
Epoch 2/80
2825/2825 [==============================] - 1447s 512ms/step - loss: 0.7119 - dense_1_loss: 0.8923 - dense_2_loss: 0.6315 - dense_3_loss: 0.4317 - dense_1_acc: 0.7578 - dense_2_acc: 0.8684 - dense_3_acc: 0.9058 - val_loss: 0.0574 - val_dense_1_loss: 0.1566 - val_dense_2_loss: 0.0570 - val_dense_3_loss: 0.0491 - val_dense_1_acc: 0.9577 - val_dense_2_acc: 0.9852 - val_dense_3_acc: 0.9865
Epoch 3/80
2825/2825 [==============================] - 1448s 512ms/step - loss: 0.7095 - dense_1_loss: 0.8888 - dense_2_loss: 0.6286 - dense_3_loss: 0.4319 - dense_1_acc: 0.7585 - dense_2_

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2825/2825 [==============================] - 1449s 513ms/step - loss: 0.6919 - dense_1_loss: 0.8608 - dense_2_loss: 0.6206 - dense_3_loss: 0.4253 - dense_1_acc: 0.7667 - dense_2_acc: 0.8753 - dense_3_acc: 0.9106 - val_loss: 0.0648 - val_dense_1_loss: 0.1540 - val_dense_2_loss: 0.0545 - val_dense_3_loss: 0.0470 - val_dense_1_acc: 0.9593 - val_dense_2_acc: 0.9864 - val_dense_3_acc: 0.9879
Epoch 13/80
2825/2825 [==============================] - 1450s 513ms/step - loss: 0.6893 - dense_1_loss: 0.8567 - dense_2_loss: 0.6212 - dense_3_loss: 0.4228 - dense_1_acc: 0.7674 - dense_2_acc: 0.8750 - dense_3_acc: 0.9116 - val_loss: 0.0608 - val_dense_1_loss: 0.1573 - val_dense_2_loss: 0.0542 - val_dense_3_loss: 0.0508 - val_dense_1_acc: 0.9589 - val_dense_2_acc: 0.9871 - val_dense_3_acc: 0.9877
Epoch 14/80
2825/2825 [==============================] - 1450s 513ms/step - loss: 0.6876 - dense_1_loss: 0.8527 - dense_2_loss: 0.6213 - dense_3_loss: 0.4237 - dense_1_acc: 0.7692 - dense_2_acc: 0.8773 - dens

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2825/2825 [==============================] - 1450s 513ms/step - loss: 0.6813 - dense_1_loss: 0.8429 - dense_2_loss: 0.6170 - dense_3_loss: 0.4224 - dense_1_acc: 0.7736 - dense_2_acc: 0.8794 - dense_3_acc: 0.9121 - val_loss: 0.1001 - val_dense_1_loss: 0.1552 - val_dense_2_loss: 0.0544 - val_dense_3_loss: 0.0497 - val_dense_1_acc: 0.9592 - val_dense_2_acc: 0.9877 - val_dense_3_acc: 0.9876
Epoch 20/80
1072/2825 [==========>...................] - ETA: 14:21 - loss: 0.6797 - dense_1_loss: 0.8406 - dense_2_loss: 0.6161 - dense_3_loss: 0.4216 - dense_1_acc: 0.7716 - dense_2_acc: 0.8769 - dense_3_acc: 0.9108

...

2825/2825 [==============================] - 1552s 549ms/step - loss: 0.6566 - dense_1_loss: 0.8035 - dense_2_loss: 0.6070 - dense_3_loss: 0.4125 - dense_1_acc: 0.7898 - dense_2_acc: 0.8875 - dense_3_acc: 0.9193 - val_loss: 0.0709 - val_dense_1_loss: 0.1610 - val_dense_2_loss: 0.0489 - val_dense_3_loss: 0.0432 - val_dense_1_acc: 0.9603 - val_dense_2_acc: 0.9880 - val_dense_3_acc: 0.9895
Epoch 49/80
 283/2825 [==>...........................] - ETA: 22:18 - loss: 0.6565 - dense_1_loss: 0.8003 - dense_2_loss: 0.6135 - dense_3_loss: 0.4122 - dense_1_acc: 0.7901 - dense_2_acc: 0.8904 - dense_3_acc: 0.9212

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2825/2825 [==============================] - 1551s 549ms/step - loss: 0.6541 - dense_1_loss: 0.7991 - dense_2_loss: 0.6057 - dense_3_loss: 0.4126 - dense_1_acc: 0.7898 - dense_2_acc: 0.8875 - dense_3_acc: 0.9205 - val_loss: 0.0641 - val_dense_1_loss: 0.1581 - val_dense_2_loss: 0.0488 - val_dense_3_loss: 0.0431 - val_dense_1_acc: 0.9598 - val_dense_2_acc: 0.9886 - val_dense_3_acc: 0.9895
Epoch 53/80
 758/2825 [=======>......................] - ETA: 18:12 - loss: 0.6536 - dense_1_loss: 0.7971 - dense_2_loss: 0.6059 - dense_3_loss: 0.4144 - dense_1_acc: 0.7884 - dense_2_acc: 0.8839 - dense_3_acc: 0.9189

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2825/2825 [==============================] - 1555s 550ms/step - loss: 0.6532 - dense_1_loss: 0.7978 - dense_2_loss: 0.6053 - dense_3_loss: 0.4119 - dense_1_acc: 0.7932 - dense_2_acc: 0.8898 - dense_3_acc: 0.9199 - val_loss: 0.0826 - val_dense_1_loss: 0.1634 - val_dense_2_loss: 0.0520 - val_dense_3_loss: 0.0460 - val_dense_1_acc: 0.9596 - val_dense_2_acc: 0.9884 - val_dense_3_acc: 0.9892
Epoch 57/80
2434/2825 [========================>.....] - ETA: 3:26 - loss: 0.6565 - dense_1_loss: 0.8027 - dense_2_loss: 0.6080 - dense_3_loss: 0.4125 - dense_1_acc: 0.7919 - dense_2_acc: 0.8879 - dense_3_acc: 0.9206